# Exploring Azure AI Services

## Provision an Azure AI Services resource

Azure AI Services are cloud-based services that encapsulate artificial intelligence capabilities you can incorporate into your applications. You can provision individual Azure AI services resources for specific APIs (for example, **Language** or **Vision**), or you can provision a single **Azure AI Services** resource that provides access to multiple Azure AI services APIs through a single endpoint and key. In this case, you'll use a single **Azure AI Services** resource.

1. Open the Azure portal at `https://portal.azure.com`, and sign in using the Microsoft account associated with your Azure subscription.
2. In the top search bar, search for *Azure AI services*, select **Azure AI Services**, and create an Azure AI services multi-service account resource with the following settings:
    - **Subscription**: *Your Azure subscription*
    - **Resource group**: *Choose or create a resource group (if you are using a restricted subscription, you may not have permission to create a new resource group - use the one provided)*
    - **Region**: *Choose any available region*
    - **Name**: *Enter a unique name*
    - **Pricing tier**: Standard S0
3. Select the required checkboxes and create the resource.
4. Wait for deployment to complete, and then view the deployment details.
5. Go to the resource and view its **Keys and Endpoint** page. This page contains the information that you will need to connect to your resource and use it from applications you develop. Specifically:
    - An HTTP *endpoint* to which client applications can send requests.
    - Two *keys* that can be used for authentication (client applications can use either key to authenticate).
    - The *location* where the resource is hosted. This is required for requests to some (but not all) APIs.

## Use a REST Interface

The Azure AI services APIs are REST-based, so you can consume them by submitting JSON requests over HTTP. In this example, you'll explore a console application that uses the **Language** REST API to perform language detection; but the basic principle is the same for all of the APIs supported by the Azure AI Services resource.


In [33]:
COG_SERVICE_ENDPOINT="https://xxxxxxx.cognitiveservices.azure.com/"
COG_SERVICE_KEY="xxxxxxxxxxxxxx"
region = "westus"

In [4]:
import os
import http.client, base64, json, urllib
from urllib import request, parse, error


In [6]:
text = input ("Enter some text")
jsonBody = {
            "documents":[
                {"id": 1,
                 "text": text}
            ]
        }

# Let's take a look at the JSON we'll send to the service
print(json.dumps(jsonBody, indent=2))

{
  "documents": [
    {
      "id": 1,
      "text": "gracias"
    }
  ]
}


In [7]:
# Make an HTTP request to the REST interface
uri = COG_SERVICE_ENDPOINT.rstrip('/').replace('https://', '')
conn = http.client.HTTPSConnection(uri)

# Add the authentication key to the request header
headers = {
    'Content-Type': 'application/json',
    'Ocp-Apim-Subscription-Key': COG_SERVICE_KEY
}

# Use the Text Analytics language API
conn.request("POST", "/text/analytics/v3.1/languages?", str(jsonBody).encode('utf-8'), headers)

# Send the request
response = conn.getresponse()
data = response.read().decode("UTF-8")

print(data)

{"documents":[{"id":"1","detectedLanguage":{"name":"Spanish","iso6391Name":"es","confidenceScore":1.0},"warnings":[]}],"errors":[],"modelVersion":"2024-04-01"}


In [8]:
# If the call was successful, get the response
if response.status == 200:

    # Display the JSON response in full (just so we can see it)
    results = json.loads(data)
    print(json.dumps(results, indent=2))

    # Extract the detected language name for each document
    for document in results["documents"]:
        print("\nLanguage:", document["detectedLanguage"]["name"])

{
  "documents": [
    {
      "id": "1",
      "detectedLanguage": {
        "name": "Spanish",
        "iso6391Name": "es",
        "confidenceScore": 1.0
      },
      "warnings": []
    }
  ],
  "errors": [],
  "modelVersion": "2024-04-01"
}

Language: Spanish


## Use an SDK

You can write code that consumes Azure AI services REST APIs directly, but there are software development kits (SDKs) for many popular programming languages, including Microsoft C#, Python, and Node.js. Using an SDK can greatly simplify development of applications that consume Azure AI services.



In [9]:
!pip install azure-ai-textanalytics==5.3.0 --quiet

In [10]:
import os
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

In [11]:
# Create client using endpoint and key
credential = AzureKeyCredential(COG_SERVICE_KEY)
client = TextAnalyticsClient(endpoint=COG_SERVICE_ENDPOINT, credential=credential)

# Call the service to get the detected language
detectedLanguage = client.detect_language(documents = [text])[0]

In [12]:
print(detectedLanguage.primary_language.name)

Spanish


# Analyze Text

In [13]:
# import namespaces
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient

In [14]:
# Create client using endpoint and key
credential = AzureKeyCredential(COG_SERVICE_KEY)
cog_client = TextAnalyticsClient(endpoint=COG_SERVICE_ENDPOINT, credential=credential)

## Detect language

In [15]:
text = "Apa Kabar!"
print(text)

Apa Kabar!


In [16]:
# Get language
detectedLanguage = cog_client.detect_language(documents=[text])[0]
print('\nLanguage: {}'.format(detectedLanguage.primary_language.name))


Language: Indonesian


## Evaluate sentiment

Sentiment analysis is a commonly used technique to classify text as positive or negative (or possible neutral or mixed). It's commonly used to analyze social media posts, product reviews, and other items where the sentiment of the text may provide useful insights.

In [17]:
text = "The movie was amazing and I loved it. Charles did a fantastic job here."
print(text)

The movie was amazing and I loved it. Charles did a fantastic job here.


In [18]:
# Get sentiment
sentimentAnalysis = cog_client.analyze_sentiment(documents=[text])[0]
print("\nSentiment: {}".format(sentimentAnalysis.sentiment))


Sentiment: positive


In [19]:
print(sentimentAnalysis.confidence_scores)

{'positive': 0.99, 'neutral': 0.01, 'negative': 0.0}


## Identify key phrases
It can be useful to identify key phrases in a body of text to help determine the main topics that it discusses.

In [28]:
text = """ In recent years, the field of artificial intelligence (AI) has seen significant advancements, particularly in the development 
of machine learning algorithms and natural language processing (NLP) techniques. 
Companies like Google, IBM, and Microsoft have been at the forefront of these innovations, investing heavily in research and development.

One of the most notable breakthroughs is the creation of deep learning models, such as convolutional neural networks (CNNs) and recurrent neural networks (RNNs). 
These models have enabled substantial improvements in image and speech recognition tasks, leading to applications in various industries, including healthcare, finance, and autonomous driving.

In healthcare, AI-powered diagnostic tools are revolutionizing the way diseases are detected and treated. For example, AI algorithms can analyze medical images to 
identify early signs of cancer, improving patient outcomes through early intervention. In finance, machine learning models are used for fraud detection and to provide personalized financial advice to customers.
"""
print(text)

 In recent years, the field of artificial intelligence (AI) has seen significant advancements, particularly in the development 
of machine learning algorithms and natural language processing (NLP) techniques. 
Companies like Google, IBM, and Microsoft have been at the forefront of these innovations, investing heavily in research and development.

One of the most notable breakthroughs is the creation of deep learning models, such as convolutional neural networks (CNNs) and recurrent neural networks (RNNs). 
These models have enabled substantial improvements in image and speech recognition tasks, leading to applications in various industries, including healthcare, finance, and autonomous driving.

In healthcare, AI-powered diagnostic tools are revolutionizing the way diseases are detected and treated. For example, AI algorithms can analyze medical images to 
identify early signs of cancer, improving patient outcomes through early intervention. In finance, machine learning models are used

In [29]:
# Get key phrases
phrases = cog_client.extract_key_phrases(documents=[text])[0].key_phrases
if len(phrases) > 0:
    print("\nKey Phrases:")
    for phrase in phrases:
        print('\t{}'.format(phrase))


Key Phrases:
	natural language processing
	convolutional neural networks
	recurrent neural networks
	speech recognition tasks
	AI-powered diagnostic tools
	personalized financial advice
	machine learning algorithms
	deep learning models
	machine learning models
	recent years
	artificial intelligence
	significant advancements
	NLP) techniques
	notable breakthroughs
	substantial improvements
	various industries
	autonomous driving
	AI algorithms
	medical images
	early signs
	patient outcomes
	early intervention
	fraud detection
	field
	development
	Companies
	Google
	IBM
	Microsoft
	forefront
	innovations
	research
	creation
	CNNs
	RNNs
	applications
	healthcare
	finance
	way
	diseases
	example
	cancer
	customers


## Extract entities

Often, documents or other bodies of text mention people, places, time periods, or other entities. The text Analytics API can detect multiple categories (and subcategories) of entity in your text.



In [30]:
text = """ On June 15, 2023, John Smith, a software engineer at Google, Inc., traveled from San Francisco to New York City for a conference on Artificial Intelligence. 
The event was held at the Javits Center and sponsored by IBM and Microsoft. 
During his stay, John stayed at the Hilton Hotel and dined at the renowned restaurant Eleven Madison Park. 
He met with Dr. Jane Doe, a leading researcher from Stanford University, to discuss advancements in natural language processing.

In his free time, John visited the Metropolitan Museum of Art and took a ferry to see the Statue of Liberty. 
He also attended a Broadway show, "Hamilton," and purchased a souvenir from a local shop. 
On the flight back, he read the latest issue of The New York Times and finished a book titled "The Innovators" by Walter Isaacson."""
print(text)

 On June 15, 2023, John Smith, a software engineer at Google, Inc., traveled from San Francisco to New York City for a conference on Artificial Intelligence. 
The event was held at the Javits Center and sponsored by IBM and Microsoft. 
During his stay, John stayed at the Hilton Hotel and dined at the renowned restaurant Eleven Madison Park. 
He met with Dr. Jane Doe, a leading researcher from Stanford University, to discuss advancements in natural language processing.

In his free time, John visited the Metropolitan Museum of Art and took a ferry to see the Statue of Liberty. 
He also attended a Broadway show, "Hamilton," and purchased a souvenir from a local shop. 
On the flight back, he read the latest issue of The New York Times and finished a book titled "The Innovators" by Walter Isaacson.


In [27]:
# Get entities
entities = cog_client.recognize_entities(documents=[text])[0].entities
if len(entities) > 0:
    print("\nEntities")
    for entity in entities:
        print('\t{} ({})'.format(entity.text, entity.category))


Entities
	artificial intelligence (Skill)
	AI (Skill)
	development (Skill)
	machine learning algorithms (Skill)
	natural language processing (Skill)
	NLP (Skill)
	Google (Organization)
	IBM (Organization)
	Microsoft (Organization)
	innovations (Skill)
	research (Skill)
	development (Skill)
	One (Quantity)
	deep learning (Skill)
	image (Skill)
	speech recognition (Skill)
	healthcare (Skill)
	finance (Skill)
	AI (Skill)
	diagnostic tools (Product)
	AI (Skill)
	algorithms (Product)
	images (Product)
	patient (PersonType)
	intervention (Skill)
	finance (Skill)
	machine learning (Skill)
	financial (Skill)
	customers (PersonType)


## Extract linked entities
In addition to categorized entities, the Text Analytics API can detect entities for which there are known links to data sources, such as Wikipedia.

In [25]:
# Get linked entities
entities = cog_client.recognize_linked_entities(documents=[text])[0].entities
if len(entities) > 0:
    print("\nLinks")
    for linked_entity in entities:
        print('\t{} ({})'.format(linked_entity.name, linked_entity.url))


Links
	June 15 (https://en.wikipedia.org/wiki/June_15)
	John Doe (https://en.wikipedia.org/wiki/John_Doe)
	Google (https://en.wikipedia.org/wiki/Google)
	San Francisco (https://en.wikipedia.org/wiki/San_Francisco)
	New York City (https://en.wikipedia.org/wiki/New_York_City)
	Artificial intelligence (https://en.wikipedia.org/wiki/Artificial_intelligence)
	Javits Center (https://en.wikipedia.org/wiki/Javits_Center)
	IBM (https://en.wikipedia.org/wiki/IBM)
	Microsoft (https://en.wikipedia.org/wiki/Microsoft)
	New York Hilton Midtown (https://en.wikipedia.org/wiki/New_York_Hilton_Midtown)
	Eleven Madison Park (https://en.wikipedia.org/wiki/Eleven_Madison_Park)
	Stanford University (https://en.wikipedia.org/wiki/Stanford_University)
	Metropolitan Museum of Art (https://en.wikipedia.org/wiki/Metropolitan_Museum_of_Art)
	Statue of Liberty (https://en.wikipedia.org/wiki/Statue_of_Liberty)
	Broadway theatre (https://en.wikipedia.org/wiki/Broadway_theatre)
	Hamilton (musical) (https://en.wikipe

# Translate Text
Azure AI Translator is a service that enables you to translate text between languages.

For example, suppose a travel agency wants to examine hotel reviews that have been submitted to the company's web site, standardizing on English as the language that is used for analysis. By using Azure AI Translator, they can determine the language each review is written in, and if it is not already English, translate it from whatever source language it was written in into English.

## Detect language
Azure AI Translator can automatically detect the source language of text to be translated, but it also enables you to explicitly detect the language in which text is written.

In [34]:
text = """ 
שלום לכולם,
מה שלומכם?
תודה"""

In [35]:
import requests
# Use the Azure AI Translator detect function

translator_endpoint = 'https://api.cognitive.microsofttranslator.com'

path = '/detect'
url = translator_endpoint + path

# Build the request
params = {
    'api-version': '3.0'
}

headers = {
'Ocp-Apim-Subscription-Key': COG_SERVICE_KEY,
'Ocp-Apim-Subscription-Region': region,
'Content-type': 'application/json'
}

body = [{
    'text': text
}]

# Send the request and get response
request = requests.post(url, params=params, headers=headers, json=body)
response = request.json()

# Parse JSON array and get language
print(response)

[{'isTranslationSupported': True, 'isTransliterationSupported': True, 'language': 'he', 'score': 1.0}]


## Translate text
Now that your application can determine the language in which reviews are written, you can use Azure AI Translator to translate any non-English reviews into English.

In [39]:
# Use the Azure AI Translator translate function
path = '/translate'
url = translator_endpoint + path

# Build the request
params = {
    'api-version': '3.0',
    'from': 'he',
    'to': ['en','id','fr','es']
}

headers = {
    'Ocp-Apim-Subscription-Key': COG_SERVICE_KEY,
    'Ocp-Apim-Subscription-Region': region,
    'Content-type': 'application/json'
}

body = [{
    'text': text
}]

# Send the request and get response
request = requests.post(url, params=params, headers=headers, json=body)
response = request.json()

# Parse JSON array and get translation
translation = response[0]["translations"][0]["text"]

In [40]:
print(translation)

 
Hello everyone,
How are you?
Thank you


In [42]:
for output in response[0]['translations']:
    print(output['text'])

 
Hello everyone,
How are you?
Thank you
 
Halo semuanya,
Bagaimana keadaanmu?
Terima kasih
 
Bonjour à tous,
Comment vas-tu?
Merci
 
Hola a todos,
¿Cómo estás?
Gracias
